# Keras implementaion for the Finger Orientaion Model

This model was not used in the paper, we model has some slight variations to the paper model. In contrast to the paper, here we use an RMSprop optimizer with a declining learning rate. Hyperparameter of the optimizer are randomly chosen and are not tuned to achieve the best result.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.__version__

'1.7.0'

In [3]:
%%time
df = pd.read_pickle('./data/all_data.pkl')

CPU times: user 2.35 s, sys: 1.47 s, total: 3.82 s
Wall time: 3.82 s


In [4]:
df.MatrixCroppedSameSize = df.MatrixCroppedSameSize.apply(lambda x: np.multiply(x, 1.0 / 255.0))

if (tf.keras.backend.image_data_format() == 'channels_last'):
    df.MatrixCroppedSameSize = df.MatrixCroppedSameSize.apply(lambda x: x.reshape(22,15,1))
else:
    df.MatrixCroppedSameSize = df.MatrixCroppedSameSize.apply(lambda x: x.reshape(1,22,15))

In [5]:
split = (80, 20)
ps = np.array(sorted(df.Participant.unique()))
num_pt = len(ps)
x1 = (int)(np.floor(len(ps) * split[0] / 100.0))
split_train = ps[:x1]
x2 = (int)(np.floor(len(ps) * split[1] / 100.0))
split_test = ps[x1:]
dfTest = df[df.Participant.isin(split_test)]
dfTrain = df[df.Participant.isin(split_train)]
print("Participants in the train data set %s" % split_train)
print("Participants in the  test data set %s" % split_test)

Participants in the train data set [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26]
Participants in the  test data set [27 28 29 30 31 32 33]


In [6]:
input_shape = dfTest.iloc[0].MatrixCroppedSameSize.shape

print("Shape of the imapge to tain: " + str(input_shape))

Shape of the imapge to tain: (22, 15, 1)


In [7]:
%%time
x_train = np.stack(dfTrain.MatrixCroppedSameSize.values, axis=0)
y_train = np.stack(dfTrain[["Pitch", "Yaw"]].values, axis=0)
x_test  = np.stack(dfTest.MatrixCroppedSameSize.values, axis=0)
y_test  = np.stack(dfTest[["Pitch", "Yaw"]].values, axis=0)

CPU times: user 1.34 s, sys: 244 ms, total: 1.58 s
Wall time: 1.58 s


In [8]:
batch_size = 100

# input image dimensions
img_rows, img_cols = 27, 15

config = tf.ConfigProto(log_device_placement=True, allow_soft_placement=True, device_count = {'GPU' : 4})
config.gpu_options.allow_growth=True
#config.gpu_options.per_process_gpu_memory_fraction=1.0
#config.gpu_options.allocator_type = 'BFC'

with tf.device('/gpu:0'):
    session = tf.Session(config=config)
    model = tf.keras.Sequential()
    
    model.add(tf.keras.layers.Conv2D(32, kernel_size=(6, 6),
                                     activation='relu',
                                     padding='same',
                                     bias_initializer=tf.keras.initializers.Constant(value=0.01),
                                     input_shape=input_shape))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2), padding='same'))
    
    model.add(tf.keras.layers.Conv2D(72, kernel_size=(6, 6),
                                     activation='relu',
                                     bias_initializer=tf.keras.initializers.Constant(value=0.01),
                                     padding='same'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2), padding='same'))
    
    model.add(tf.keras.layers.Conv2D(160, kernel_size=(7, 7),
                                     activation='relu',
                                     bias_initializer=tf.keras.initializers.Constant(value=0.01),
                                     padding='same'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2), padding='same'))
    
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(2000, 
            activation='relu',
            kernel_initializer=tf.keras.initializers.glorot_uniform(),
            bias_initializer=tf.keras.initializers.Constant(value=0.01),
            kernel_regularizer=tf.keras.regularizers.l2(0.15)))
    
    model.add(tf.keras.layers.Dense(2,
            kernel_initializer=tf.keras.initializers.glorot_uniform(),
            bias_initializer=tf.keras.initializers.Constant(value=0.01),
            kernel_regularizer=tf.keras.regularizers.l2(0.15)))
    
    optimizer = tf.keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.1)
    
    print(model.summary())
    model.compile(loss=tf.keras.losses.mean_squared_error,
                  optimizer=optimizer)
    
    # Function to display the target and prediciton
    def testmodel(epoch, logs):
        pred_y = model.predict(
            x_test,
            batch_size=batch_size
        )
        true_y = y_test
        
        ePitch = np.abs(true_y[:,0] - pred_y[:,0]) 
        eYaw = np.abs(true_y[:,1] - pred_y[:,1])
        ListError = ePitch + eYaw
        ListErrorPitch = ePitch
        ListErrorYaw = eYaw

        diff_true = true_y[:,0] + true_y[:,1]
        diff_pred = pred_y[:,0] + pred_y[:,1]
        RMSE = np.sqrt(np.mean(np.square(np.subtract(diff_pred, diff_true))))
        RMSEPitch = np.sqrt(np.mean(np.square(np.subtract(pred_y[:,0], true_y[:,0]))))
        RMSEYaw   = np.sqrt(np.mean(np.square(np.subtract(pred_y[:,1], true_y[:,1]))))
        
        print ('RMSE: %.2f RMSE-P: %.2f RMSE-Y: %.2f e: %.2f E-P: %.2f E-Y: %.2f std: %.2f std-P: %.2f std-Y: %.2f' %(
                  RMSE, RMSEPitch, RMSEYaw,
                    ListError.mean(), ListErrorPitch.mean(), ListErrorYaw.mean(),
                    ListError.std(), ListErrorPitch.std(), ListErrorYaw.std(),))

    
    # Callback to display the target and prediciton
    testcallback = tf.keras.callbacks.LambdaCallback(on_epoch_end=testmodel)

    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=50,
              verbose=1,
              validation_data=(x_test, y_test),
              callbacks=[testcallback])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 22, 15, 32)        1184      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 8, 32)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 8, 72)         83016     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 4, 72)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 6, 4, 160)         564640    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 3, 2, 160)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 960)               0         
__________

368836/368836 [==============================] - 122s 330us/step - loss: 936.9019 - val_loss: 1058.8702
RMSE: 44.80 RMSE-P: 15.78 RMSE-Y: 42.32 e: 47.20 E-P: 13.08 E-Y: 34.11 std: 26.68 std-P: 8.82 std-Y: 25.04
Epoch 32/50
368836/368836 [==============================] - 121s 328us/step - loss: 934.0718 - val_loss: 1055.5248
RMSE: 44.73 RMSE-P: 15.77 RMSE-Y: 42.24 e: 47.11 E-P: 13.08 E-Y: 34.03 std: 26.66 std-P: 8.82 std-Y: 25.03
Epoch 33/50
368836/368836 [==============================] - 122s 330us/step - loss: 931.3946 - val_loss: 1052.9099
RMSE: 44.65 RMSE-P: 15.79 RMSE-Y: 42.17 e: 47.04 E-P: 13.09 E-Y: 33.95 std: 26.66 std-P: 8.82 std-Y: 25.02
Epoch 34/50
368836/368836 [==============================] - 123s 333us/step - loss: 928.9233 - val_loss: 1049.8080
RMSE: 44.61 RMSE-P: 15.78 RMSE-Y: 42.10 e: 46.95 E-P: 13.08 E-Y: 33.87 std: 26.64 std-P: 8.82 std-Y: 25.00
Epoch 35/50
368836/368836 [==============================] - 121s 328us/step - loss: 926.5872 - val_loss: 1047.8299
RMSE